In [261]:
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import re
from collections import Counter
from collections import defaultdict
import copy

from scipy.sparse import csr_matrix
from sklearn.naive_bayes import BernoulliNB

from  math import exp
import random










In [447]:
df1 = pd.read_csv("train.csv")
df2 = df1[df1.Insult == 0]
df3 = df1[df1.Insult == 1]
df0 = df1

lst_of_0_token = tokenize(df2.Comment)
counter_0 = Counter(lst_of_0_token)

lst_of_1_token = tokenize(df3.Comment)
counter_1 = Counter(lst_of_1_token)

lst_of_all_token = tokenize(df1.Comment)
counter_all = Counter(lst_of_all_token)




In [448]:
print(len(counter_all))

24882


In [449]:


#     print(text)
def tokenize(dataframe):
    lst_of_token = []
    for i in dataframe:
        opt = i.lower()
        opt = opt.replace('"', '',1)
        opt = opt.replace('"',' ', -1)
        text = opt.split()
        lst_of_token += text
    return lst_of_token
# # print(type(text))
# # all_df = df1
# lst_of_all_token = tokenize(all_df.Comment[:5])
# # df2 = df1[df1.Insult == 1]
# lst_of_1_token = tokenize(df2.Comment[:5])
    
# print(lst_of_1_token)
def tokenstring(i):
    lst_of_token = []
    opt = i.lower()
    opt = opt.replace('"', '',1)
    opt = opt.replace('"',' ', -1)
    text = opt.split()
    lst_of_token += text
    return lst_of_token



In [450]:
def sigmoid(X):
    
    result = 1 / (1+ math.exp(-X))
    return result


In [451]:
all_dict = counter_all
all_dict = dict.fromkeys(all_dict,0)
# print(all_dict)
all_dict['BIAS'] = 1

In [452]:

def make_comment_dict_lst():
    lst_of_comment_dict = []
    for i in df1.Comment:
        lst_of_comment = tokenstring(i)
        count_comment = Counter(lst_of_comment)
        count_comment['BIAS'] = 1
        lst_of_comment_dict.append(count_comment)
#     print(lst_of_comment_dict[:1])
    return lst_of_comment_dict

In [453]:
comment_dict_lst = make_comment_dict_lst()

In [454]:
b = np.zeros(len(all_dict))
# sparse_b = csr_matrix(b)
default_b = b.T
default_b

array([0., 0., 0., ..., 0., 0., 0.])

In [455]:

def make_row_matrix(lst):

    row_array_dict = {}
    n = 0
    for i in lst:
        new_array_list = []
        all_dict = counter_all
        all_dict = dict.fromkeys(all_dict,0)
        all_dict['BIAS'] = 1
        for word in dict(i):
    #         print(word)
            all_dict[word] = i[word]
        new_array = np.array(list(all_dict.values()))
    #     print(new_array)
        sparse_matrix = csr_matrix(new_array)
    #     print(sparse_matrix)
        row_array_dict[n] = sparse_matrix
        n += 1
#         print(array_dict)
    return row_array_dict


In [456]:
row_array_dict = make_row_matrix(comment_dict_lst)


In [457]:
# print(row_array_dict)

In [458]:
test_dict = {}
test_dict[0] = row_array_dict[0]
print(test_dict)

{0: <1x24883 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>}


In [461]:
def make_X_dict(array_dict, b):
    X_dict = {}
    n = 0
    for array in array_dict.values():
        sumxb = array.dot(b)
        X_dict[n]  =sumxb
        n+=1
    return X_dict

    

In [462]:
x_test = make_X_dict(test_dict,default_b)

In [463]:
def make_true_value():
    true_lst = []
    for row in df1.Insult:
        true_lst.append(row)
    return true_lst


In [464]:
true = make_true_value()

In [465]:
def Likelihood(array, index, test_b):
    sumxb = array.dot(test_b)
#     print(type(sumxb))
    sig  = sigmoid(sumxb)
#     diff = true[index] - sig
    return sig



In [466]:
def FindF1(beta):
    index = 0
    pro = []
    for i in row_array_dict.values():
        sig = Likelihood(i,index, beta)
        if sig > 0.5:
            pro.append(1)
        else:
            pro.append(0)
    f1 = f1_score(true, pro, average='micro')
            
    
    
    
    
    return f1

# print(FindF1(default_b))
    

In [467]:
def descent(alpha):
    b = np.zeros((len(all_dict),1))
    round_num = 0
    F1 = 0
    while round_num < 89:
#         i = numpy.random.random_integers(len(df1.Comment)-1)
        print("round", round_num)
        for loop in range(len(df1.Comment)):
#             print("loop", i)
            i = numpy.random.random_integers(len(df1.Comment)-1)
            like = Likelihood(row_array_dict[i], i, b)

#             print(like)
            ara = alpha *(true[i] - like)*(row_array_dict[i].T)
#             print(ara)
    #         print(type(b))
            b = b + ara
        F1 = FindF1(b)
        print(F1)
        round_num += 1
    #         print(b)
#             if abs(FindF1(b) - F1 ) > 0.1:
#                 F1 = FindF1(b)
#                 print(F1)
    return b,F1

In [468]:
descent(0.01)

round 0


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(1, 3946 + 1) instead
  # Remove the CWD from sys.path while we load stuff.


0.7785659994932861
round 1
0.7945274892323283
round 2
0.827717253610337
round 3
0.8358246769698505
round 4
0.8406384595895617
round 5
0.8484925259690904
round 6


KeyboardInterrupt: 

In [368]:
dev_df = pd.read_csv('dev.csv')

In [369]:
dev_df.head()

,Insult,Comment
0,0,"""like this if you are a tribe fan"""
1,1,"""you're idiot......................."""
2,1,"""I am a woman Babs, and the only ""war on women..."
3,1,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F..."
4,1,"""haha green me red you now loser whos winning ..."
